<a href="https://colab.research.google.com/github/mangare91/Drug_Discover_Portifolio_projects/blob/main/project1_1_importing%26preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing libraries**

In [1]:
! pip install chembl_webresource_client

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for chembl-webresource-client: filename=chembl_webresource_client-0.10.2-cp36-none-any.whl size=55662 sha256=13021702895e2a4cb5d9fbf7a988b6b92fba5dda7d38036ede6d74c6923d00b8
  Stored in directory: /root/.cache/pip/wheels/e6/96/19/3f042bfda7c669bfe24c8278477f57b0fbbf3e488b4c09e3a8
Successfully built chembl-webresource-client


## **Importing libraries**

In [2]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for coronavirus**

In [3]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)
targets

cross_references  ...   tax_id
0                                                 []  ...    11119
1                                                 []  ...   227859
2                                                 []  ...    12663
3                                                 []  ...    11137
4  [{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...  ...   227859
5                                                 []  ...  1335626
6  [{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...  ...   227859

[7 rows x 9 columns]

### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

In [4]:
selected_target = targets.target_chembl_id[4]
selected_target

'CHEMBL3927'

In [5]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [6]:
df = pd.DataFrame.from_dict(res)

In [7]:
df.head(3)

activity_comment  activity_id  ... upper_value value
0             None      1480935  ...        None   7.2
1             None      1480936  ...        None   9.4
2             None      1481061  ...        None  13.5

[3 rows x 43 columns]

In [8]:
df.to_csv('bioactivity_data_raw.csv', index=False)

## **Handling missing data**

In [9]:
df2 = df[df.standard_value.notna()]
df2

activity_comment  activity_id  ... upper_value  value
0               None      1480935  ...        None    7.2
1               None      1480936  ...        None    9.4
2               None      1481061  ...        None   13.5
3               None      1481065  ...        None  13.11
4               None      1481066  ...        None    2.0
..               ...          ...  ...         ...    ...
128             None     12041507  ...        None   10.6
129             None     12041508  ...        None   10.1
130             None     12041509  ...        None   11.5
131             None     12041510  ...        None   10.7
132             None     12041511  ...        None   78.9

[133 rows x 43 columns]

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**

In [10]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [11]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3

molecule_chembl_id  ... standard_value
0         CHEMBL187579  ...         7200.0
1         CHEMBL188487  ...         9400.0
2         CHEMBL185698  ...        13500.0
3         CHEMBL426082  ...        13110.0
4         CHEMBL187717  ...         2000.0
..                 ...  ...            ...
128      CHEMBL2146517  ...        10600.0
129       CHEMBL187460  ...        10100.0
130       CHEMBL363535  ...        11500.0
131       CHEMBL227075  ...        10700.0
132        CHEMBL45830  ...        78900.0

[133 rows x 3 columns]

In [12]:
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

molecule_chembl_id  ... bioactivity_class
0         CHEMBL187579  ...      intermediate
1         CHEMBL188487  ...      intermediate
2         CHEMBL185698  ...          inactive
3         CHEMBL426082  ...          inactive
4         CHEMBL187717  ...      intermediate
..                 ...  ...               ...
128      CHEMBL2146517  ...          inactive
129       CHEMBL187460  ...          inactive
130       CHEMBL363535  ...          inactive
131       CHEMBL227075  ...          inactive
132        CHEMBL45830  ...          inactive

[133 rows x 4 columns]

In [13]:
df4.to_csv('bioactivity_data_preprocessed.csv', index=False)

---